In [2]:
import psycopg2
import gspread
import json
import os
from datetime import datetime
from oauth2client.service_account import ServiceAccountCredentials

# ===== Read secrets from GitHub Actions =====
PGHOST = os.getenv("PGHOST")
PGDATABASE = os.getenv("PGDATABASE")
PGUSER = os.getenv("PGUSER")
PGPASSWORD = os.getenv("PGPASSWORD")
PGPORT = os.getenv("PGPORT")
SPREADSHEET_URL = os.getenv("SPREADSHEET_URL")
GOOGLE_CREDS = os.getenv("GOOGLE_CREDS")  # JSON string from GitHub Secrets

# ===== Connect to Google Sheets =====
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds_dict = json.loads(GOOGLE_CREDS)
creds = ServiceAccountCredentials.from_json_keyfile_dict(creds_dict, scope)
client = gspread.authorize(creds)

# ===== Export function =====
def export_to_google_sheets():
    try:
        conn = psycopg2.connect(
            dbname=PGDATABASE,
            user=PGUSER,
            password=PGPASSWORD,
            host=PGHOST,
            port=PGPORT
        )
        cur = conn.cursor()
        # Fetch all rows from the table
        cur.execute("SELECT * FROM ecommerce_sales;")
        data = cur.fetchall()
        col_names = [desc[0] for desc in cur.description]

        # Update Google Sheet all at once
        sheet = client.open_by_url(SPREADSHEET_URL).sheet1
        sheet.clear()
        sheet.update([col_names] + list(data))

        print(f"[{datetime.now()}] Exported {len(data)} rows to Google Sheets.")

    except Exception as e:
        print(f"[{datetime.now()}] Error: {e}")

    finally:
        if conn:
            conn.close()

if __name__ == "__main__":
    export_to_google_sheets()


TypeError: the JSON object must be str, bytes or bytearray, not NoneType